**MODELO DE MACHINE LEARNING CON PROCESAMIENTO DE LENGUAJE NATURAL**

Las reseñas ofrecen información valiosa al proporcionar opiniones directas de los clientes sobre diversos aspectos del negocio, lo que facilita la obtención de conocimientos clave para mejorar los servicios y tomar decisiones estratégicas. 
El objetivo principal de este proyecto de Machine Learning es extraer información valiosa de las reseñas de clientes para China Garden, centradas en las siguientes categorías: comida, servicio, precio y ubicación. Dado el elevado volumen de comentarios, realizar este análisis de forma manual sería un proceso ineficiente y laborioso. Para enfrentar este desafío, se etiquetarán manualmente un conjunto representativo de reseñas que cubran las diferentes categorías, las cuales servirán como base para entrenar un modelo de reconocimiento de entidades mediante la biblioteca spaCy.
Una vez identificadas las categorías, se realizará un análisis de sentimiento con la herramienta VADER. Además, spaCy se utilizará para extraer los adjetivos y adverbios asociados a las distintas etiquetas, con el objetivo de evaluar el nivel de satisfacción de los clientes en cada categoría y proporcionar una interpretación lingüística que refleje su percepción general.  


*Para desarrollar el producto seguiremos las siguientes etapas:*

1.	Preprocesamiento: Adquisición de datos y limpieza del texto, adaptando el mismo al formato necesario para el entrenamiento del modelo.
2.	Ingeniería de características (Feature engineering): Extracción de las características clave mediante el entrenamiento de un modelo de Machine Learning.
3.	Evaluación del modelo: Evaluación del rendimiento utilizando métricas adecuadas para asegurar su precisión.
4.	Análisis de sentimientos: Evaluación del sentimiento asociado a cada categoría, con especial énfasis en la identificación de adjetivos y adverbios que describen cada aspecto.
5.	Despliegue (Deployment): Implementación del modelo para su uso en producción, garantizando su funcionamiento en un entorno real.


Para analizar y extraer esta información utilizaremos **spaCy**, una potente biblioteca de procesamiento de lenguaje natural (NLP) en Python. spaCy ofrece una variedad de herramientas eficientes para trabajar con texto en lenguaje humano, incluyendo NER, para la identificación de entidades. Además, está diseñada para manejar grandes volúmenes de datos, lo que la hace ideal para proyectos de producción. Su capacidad de personalización permite adaptar modelos preentrenados o entrenar nuevos modelos específicos para casos concretos, como la extracción de características clave en reseñas.

In [1]:
#importamos las bibliotecas requeridas

import re 
import pandas as pd
import spacy
from spacy import displacy
from spacy.training import Example
from spacy.pipeline import Tagger
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 

In [4]:
nlp  = spacy.blank("en") #cargamos un modelo spaCY en blanco que 'entiende' inglés 
#nlp.pipe_names

# Añadimos el componente de NER (Named Entity Recognition,) al pipeline si no está presente
if "ner" not in nlp.pipe_names:
    ner = nlp.add_pipe('ner', last=True)
else:
    ner = nlp.get_pipe("ner")

# Añadimos las etiquetas al NER
labels = ['SERVICIO', 'COMIDA', 'PRECIO', 'LOCACIÓN'] 
for label in labels:
    ner.add_label(label) 

Para la generación de los datos de ejemplo, se utilizó **Label Studio**, una herramienta de código abierto diseñada para la anotación de datos, ampliamente empleada en proyectos de aprendizaje automático. Esta plataforma permite a los usuarios etiquetar y clasificar diversos tipos de datos, como texto, imágenes, audio y video, para crear conjuntos de datos anotados que luego se utilizan en el entrenamiento de modelos de Machine Learning.

Una vez completado el proceso de etiquetado, los datos se exportan en formato CSV, que posteriormente se carga utilizando pandas para su procesamiento y análisis.

In [10]:
#cargamos el dataset generado en label-studio
path= r'C:/Users/LENOVO/Documents/Henry/Proyecto Grupal Final/ML #2SPRINT/Datos entrenar modelo NER/Juan Sample.csv'

df_revtrain = pd.read_csv(path)
df_revtrain = df_revtrain.dropna()
df_revtrain = df_revtrain.reset_index()

#eliminamos columnas inncesarias

df_revtrain= df_revtrain.drop(['annotation_id', 'annotator', 'city', 'created_at', 'id', 
                               'lead_time', 'name_x', 'name_y', 'nombre_estado', 'rating', 'updated_at', 'Unnamed: 0', 'index'], axis=1)

In [16]:
df_revtrain

,label,text
0,"[{""start"":23,""end"":40,""text"":""Very good servic...",(Translated by Google) Very good service\r\n\r...
1,"[{""start"":0,""end"":9,""text"":""Good food"",""labels...",Good food and friendly staff
2,"[{""start"":18,""end"":39,""text"":""Price is up ther...",Food is so tasty! Price is up there too. Seafo...
3,"[{""start"":29,""end"":59,""text"":""Friendly, fast a...","Great family owned business. Friendly, fast an..."
4,"[{""start"":0,""end"":12,""text"":""Amazing food"",""la...",Amazing food and service. This is not your run...
...,...,...
319,"[{""start"":0,""end"":43,""text"":""Don't expect napk...",Don't expect napkins or forks with delivery.. ...
320,"[{""start"":0,""end"":39,""text"":""The establishment...",The establishment need a good cleaning.
321,"[{""start"":0,""end"":55,""text"":""Decent but medioc...",Decent but mediocre at best egg rolls were not...
322,"[{""start"":0,""end"":78,""text"":""I got dinner here...",I got dinner here last night for the first tim...


Los datos exportados de Label-Studio vienen con dos columnas de interés: text and label.

Un ejemplo de la columna label es: **[{"start":18,"end":39,"text":"Price is up there too","labels":["PRECIO"]},{"start":0,"end":16,"text":"Food is so tasty","labels":["COMIDA"]},{"start":41,"end":63,"text":"Seafood heaven yum yum","labels":["COMIDA"]}]**.

Un ejemplo dela columna text es: **'Food is so tasty! Price is up there too. Seafood heaven yum yum'.**

Teniendo en cuenta los anteriores ejemplos, el dato que sería correcto para el entrenamiento del modelo en spaCY sería:
**('Food is so tasty! Price is up there too. Seafood heaven yum yum', { "entities": [(18, 39, "PRECIO"), (0, 16, " COMIDA "), (41, 63, "COMIDA")] }).**

Las siguientes son las transformaciones necesarias para poder crear los datos de entrenamiento en el formato correcto de spaCY utilizando la información entregada por **Label-Studio**: 

In [2]:

#ahora debemos transformar la información en un formato de entrenamiento que acepte spacy
#corregimos los labels

def extract_labels(texto):
    # Expresión regular para capturar start, end y labels 
    regex = r'"start":(\d+),"end":(\d+).*?"labels":\["([^"]+)"\]'
    
    # Buscamos todas las coincidencias
    matches = re.findall(regex, texto)
    
    # Convertimos las coincidencias en una lista de tuplas
    result = [(int(start), int(end), label) for start, end, label in matches]
    
    return result

# Aplicamos la función a la columna 'label' y creamos la nueva columna 'corrected_label'
df_revtrain['corrected_label'] = df_revtrain['label'].apply(extract_labels)

df_revtrain['corrected_format'] = [
    (row, {'entities': df_revtrain['corrected_label'][i]})
    for i, row in enumerate(df_revtrain['text'])]      

TRAIN_DATA = df_revtrain['corrected_format'].tolist()



def remove_overlapping_entities(entities):
    """Elimina entidades superpuestas de una lista de entidades."""
    non_overlapping_entities = []
    sorted_entities = sorted(entities, key=lambda x: x[0])  # Ordenamos por el inicio de la entidad
    last_end = -1
    for start, end, label in sorted_entities:
        if start >= last_end:
            non_overlapping_entities.append((start, end, label))
            last_end = end
    return non_overlapping_entities
#corregimos overlaps en los tags

def remove_overlapping_entities_from_train_data(train_data):
    """Elimina entidades superpuestas de los ejemplos en TRAIN_DATA."""
    clean_train_data = []
    
    for text, annotation in train_data:
        entities = annotation['entities']
        # Eliminamos las entidades superpuestas usando la función previamente generada
        non_overlapping_entities = remove_overlapping_entities(entities)
        # Añadimos el texto con las entidades no superpuestas
        clean_train_data.append((text, {'entities': non_overlapping_entities}))
    
    return clean_train_data

# Aplicamos la función a TRAIN_DATA
TRAIN_DATA = remove_overlapping_entities_from_train_data(TRAIN_DATA)

In [3]:
def preprocess_data(data):
    processed_data = []
    for text, annotations in data:
        # Eliminamos espacios extra y caracteres especiales
        text = re.sub(r'\s+', ' ', text).strip()
        text = re.sub(r'[^\w\s]', '', text)
        
        # Ajustamos las anotaciones
        new_entities = []
        for start, end, label in annotations['entities']:
            # Recalculamos las posiciones de inicio y fin
            new_start = len(re.sub(r'\s+', ' ', text[:start]).strip())
            new_end = len(re.sub(r'\s+', ' ', text[:end]).strip())
            if new_start < new_end:
                new_entities.append((new_start, new_end, label))
        
        processed_data.append((text, {'entities': new_entities}))
    
    return processed_data

In [4]:
TRAIN_DATA = preprocess_data(TRAIN_DATA)

In [4]:
#Teniendo los labels, construimos el formato necesario de los datoss de entrenamiento
#Inicializamos el optimizador para entrenar el modelo
optimizer = nlp.begin_training()

train_data, test_data = train_test_split(TRAIN_DATA, test_size=0.2, random_state=42)

# Entrenamos el modelo
for epoch in range(100):  # Ajusta el número de épocas según sea necesario
    losses = {}
    for text, annotations in train_data:
        # Creamos un ejemplo de entrenamiento
        example = Example.from_dict(nlp.make_doc(text), annotations)
        # Actualizamos el modelo con el texto y las entidades etiquetadas
        nlp.update([example], losses=losses, drop=0.5, sgd=optimizer)
    print(f"Epoch {epoch} - Losses: {losses}")

# Guardamos el modelo entrenado
nlp.to_disk("./modelo_ner_comida_experiencia")

Epoch 0 - Losses: {'ner': 1344.692492577242}
Epoch 1 - Losses: {'ner': 1261.4648523602757}
Epoch 2 - Losses: {'ner': 1452.399630404626}
Epoch 3 - Losses: {'ner': 1378.8031227727422}
Epoch 4 - Losses: {'ner': 1572.1595495222061}
Epoch 5 - Losses: {'ner': 1412.8476510245725}
Epoch 6 - Losses: {'ner': 1723.2561719725666}
Epoch 7 - Losses: {'ner': 1306.5338970664238}
Epoch 8 - Losses: {'ner': 1740.5350501066728}
Epoch 9 - Losses: {'ner': 1662.3123496458907}
Epoch 10 - Losses: {'ner': 1441.9062242547802}
Epoch 11 - Losses: {'ner': 1372.3655414649154}
Epoch 12 - Losses: {'ner': 1449.9473143225011}
Epoch 13 - Losses: {'ner': 1374.381743515764}
Epoch 14 - Losses: {'ner': 1430.0124955183276}
Epoch 15 - Losses: {'ner': 1465.4013213157668}
Epoch 16 - Losses: {'ner': 1336.7411863399825}
Epoch 17 - Losses: {'ner': 1431.0048039654837}
Epoch 18 - Losses: {'ner': 1288.5141685205108}
Epoch 19 - Losses: {'ner': 1262.5879679073803}
Epoch 20 - Losses: {'ner': 1251.585338815131}
Epoch 21 - Losses: {'ner': 

In [5]:
def evaluate_model(nlp, test_data):
    correct_predictions = 0
    total_predictions = 0
    for text, annotations in test_data:
        # Procesamos el texto con el modelo entrenado
        doc = nlp(text)
        # Extraemos las entidades predichas
        predicted_entities = [(ent.text, ent.label_) for ent in doc.ents]
        # Extraemos las entidades reales (etiquetas correctas)
        true_entities = [(text[start:end], label) for (start, end, label) in annotations['entities']]
        
        # Comparamos entidades predichas con las verdaderas
        correct_predictions += sum([1 for ent in predicted_entities if ent in true_entities])
        total_predictions += len(true_entities)
    
    # Calculamos la precisión
    precision = correct_predictions / total_predictions if total_predictions > 0 else 0
    print(f"Precisión del modelo: {precision * 100:.2f}%")

# Evaluamos el modelo entrenado en los datos de prueba
evaluate_model(nlp, test_data)


Precisión del modelo: 25.35%


In [6]:
# Cargamos el modelo entrenado
nlp = spacy.load("./modelo_ner_comida_experiencia")

# Probamos el modelo en un nuevo texto
doc = nlp("Ramen was good. The service was excelent. Sushi can be better.")
for ent in doc.ents:
    print(ent.text, ent.label_)

Ramen was good. COMIDA
The service was excelent. SERVICIO
Sushi can be better. COMIDA


### Implementación del modelo para la identificación de aspectos en los comentarios

Una vez que nuestro modelo NER ha sido entrenado y evaluado, lo utilizamos para identificar aspectos claves en los comentarios de los clientes, como COMIDA, LOCACIÓN, PRECIO, y SERVICIO. En este caso, trabajamos con un subconjunto aleatorio de 8000 comentarios extraídos del dataset original. El objetivo es extraer las etiquetas que el modelo encuentra dentro de los comentarios y organizarlas en columnas separadas para facilitar el análisis.

Para esto, hemos creado una función que procesa cada comentario usando el modelo NER. Esta función genera cuatro nuevas columnas en nuestro DataFrame (una para cada aspecto mencionado), donde se almacenarán las oraciones que contienen las etiquetas identificadas por el modelo.

Si un comentario incluye múltiples menciones a un aspecto específico (por ejemplo, varias menciones sobre la comida en una misma review), las diferentes menciones se separan dentro de la columna correspondiente mediante una barra vertical "|".

In [2]:
# llamamos a nuestro modelo entrenado 
nlp = spacy.load("./modelo_ner_comida_experiencia")

# Añadimos el componente 'sentencizer' al pipeline
if 'sentencizer' not in nlp.pipe_names:
    nlp.add_pipe('sentencizer', before='ner')

def process_review(review):
    doc = nlp(review)
    results = {
        'COMIDA': [],
        'LOCACIÓN': [],
        'PRECIO': [],
        'SERVICIO': []
    }
    
    for sent in doc.sents:
        for ent in sent.ents:
            if ent.label_ in results:
                # Agregamos la oración completa que contiene la entidad
                results[ent.label_].append(sent.text.strip())
    
    # Convertimos las listas a strings, manteniendo oraciones únicas
    for key in results:
        results[key] = ' | '.join(set(results[key])) if results[key] else ''
    
    return results


In [3]:
# Creamos el dataFrame 
df = pd.read_csv(r'C:\Users\ADMIN\Desktop\Practica PROYECTO FINAL\Columnas\sample_reviews.csv')

# Aplicamos la función a la columna de "text"
processed_reviews = df['text'].apply(process_review)

# Creamos de nuevas columnas en el dataFrame
for category in ['COMIDA', 'LOCACIÓN', 'PRECIO', 'SERVICIO']:
    df[category] = processed_reviews.apply(lambda x: x[category])

df

,name_x,nombre_estado,city,name_y,rating,text,COMIDA,LOCACIÓN,PRECIO,SERVICIO
0,Pacific Grove,Massachusetts,Wilmington,Lynn,4,"Very, very good meals. 13 in our party and all...",,"Very, very good meals.",There was 1 employee making sushi and a fair a...,"Very, very good meals."
1,Confucio Express Brickell,Florida,Miami,Jose David Roa Licenciado en Musica,5,The best Chinese food of Miami I strongly reco...,The best Chinese food of Miami I strongly reco...,,,
2,Chinese Chef,Virginia,Henrico,Uma Vadlamani,5,My favorite Chinese food place.,My favorite Chinese food place.,,,
3,Hardee,New York,Brooklyn,wes wright,5,Not your average Chinese spot. Fast service also,Not your average Chinese spot.,,,Fast service also
4,Szechuan Delight,New York,Brooklyn,Ralph Parrilla,5,Excellent service very good food could do with...,Excellent service very good food could do with...,,,Excellent service very good food could do with...
...,...,...,...,...,...,...,...,...,...,...
7995,YummyYummy,South Carolina,Great Falls,Scot Simons,4,Food was good and customer service was great!,Food was good and customer service was great!,,,Food was good and customer service was great!
7996,China Kitchen,Delaware,Dover,James Cox,4,"Food was fresh and tasty, I'd say it's above a...","Food was fresh and tasty, I'd say it's above a...",,,
7997,Great Wall Restaurant,South Carolina,Georgetown,Rauf Bolden,5,Excellent chinese cuisine with friendly staff.,,Excellent chinese cuisine with friendly staff.,,
7998,El Palacio Buffet,Florida,Orlando,Regina A,3,Although diverse Hispanic and Oriental cuisine...,Although diverse Hispanic and Oriental cuisine...,Although diverse Hispanic and Oriental cuisine...,,"Very busy, kind staff and a nice variety of op..."


### Implementación de VADER para el análisis de sentimientos

Tras generar el DataFrame con las 4 columnas correspondientes a los aspectos **COMIDA, LOCACIÓN, PRECIO y SERVICIO** a partir de nuestro modelo NER entrenado, implementamos el análisis de sentimientos utilizando la librería VADER. VADER es ampliamente utilizada para analizar texto, especialmente en redes sociales, debido a su capacidad para interpretar el tono de los comentarios y manejar elementos como emojis, que suelen ser relevantes en este tipo de interacciones.

El análisis se aplicará a las 4 columnas que contienen los aspectos previamente identificados en los comentarios, y se crearán otras 4 columnas adicionales con los resultados de VADER. VADER asigna un puntaje de sentimiento que varía en un rango de -1 a 1, donde:

* Valores negativos (< 0) indican un sentimiento negativo.
* Valores positivos (> 0) reflejan un sentimiento positivo.
* Cuanto más cercano esté el valor a 1, más positivo será el sentimiento; y cuanto más cercano a -1, más negativo.

Esta implementación nos permitirá medir cómo se perciben los diferentes aspectos (COMIDA, LOCACIÓN, PRECIO, SERVICIO) en cada comentario y obtener una visión detallada del sentimiento general.


In [4]:
# Copiamos el dataFrame en una nueva variable
df_vader = df.copy()

In [5]:
# Implementamos la librería Vader 
sia = SentimentIntensityAnalyzer()

def analyze_sentiment(text):
    if pd.isna(text) or text == '':
        return float(0.0)
    return sia.polarity_scores(text)['compound']

# Aplicamos el análisis de sentimiento a cada columna
for category in ['COMIDA', 'LOCACIÓN', 'PRECIO', 'SERVICIO']:
    df_vader[f'{category}_SENTIMENT'] = df_vader[category].apply(analyze_sentiment)

df_vader

,name_x,nombre_estado,city,name_y,rating,text,COMIDA,LOCACIÓN,PRECIO,SERVICIO,COMIDA_SENTIMENT,LOCACIÓN_SENTIMENT,PRECIO_SENTIMENT,SERVICIO_SENTIMENT
0,Pacific Grove,Massachusetts,Wilmington,Lynn,4,"Very, very good meals. 13 in our party and all...",,"Very, very good meals.",There was 1 employee making sushi and a fair a...,"Very, very good meals.",0.0000,0.5379,0.3182,0.5379
1,Confucio Express Brickell,Florida,Miami,Jose David Roa Licenciado en Musica,5,The best Chinese food of Miami I strongly reco...,The best Chinese food of Miami I strongly reco...,,,,0.7964,0.0000,0.0000,0.0000
2,Chinese Chef,Virginia,Henrico,Uma Vadlamani,5,My favorite Chinese food place.,My favorite Chinese food place.,,,,0.4588,0.0000,0.0000,0.0000
3,Hardee,New York,Brooklyn,wes wright,5,Not your average Chinese spot. Fast service also,Not your average Chinese spot.,,,Fast service also,0.0000,0.0000,0.0000,0.0000
4,Szechuan Delight,New York,Brooklyn,Ralph Parrilla,5,Excellent service very good food could do with...,Excellent service very good food could do with...,,,Excellent service very good food could do with...,0.9509,0.0000,0.0000,0.9509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,YummyYummy,South Carolina,Great Falls,Scot Simons,4,Food was good and customer service was great!,Food was good and customer service was great!,,,Food was good and customer service was great!,0.8070,0.0000,0.0000,0.8070
7996,China Kitchen,Delaware,Dover,James Cox,4,"Food was fresh and tasty, I'd say it's above a...","Food was fresh and tasty, I'd say it's above a...",,,,0.6239,0.0000,0.0000,0.0000
7997,Great Wall Restaurant,South Carolina,Georgetown,Rauf Bolden,5,Excellent chinese cuisine with friendly staff.,,Excellent chinese cuisine with friendly staff.,,,0.0000,0.7845,0.0000,0.0000
7998,El Palacio Buffet,Florida,Orlando,Regina A,3,Although diverse Hispanic and Oriental cuisine...,Although diverse Hispanic and Oriental cuisine...,Although diverse Hispanic and Oriental cuisine...,,"Very busy, kind staff and a nice variety of op...",0.3612,0.3612,0.0000,0.7564


### Identificación de adjetivos y adverbios en los comentarios

Después de realizar el análisis de sentimientos para cada una de las columnas **(COMIDA, PRECIO, LOCACIÓN, SERVICIO)**, utilizaremos un modelo preentrenado de la librería **SpaCy**, llamado **"en_core_web_sm"**, para identificar los **adjetivos** y **adverbios** que aparecen en los comentarios relacionados con estos aspectos.

El objetivo es extraer los adjetivos y adverbios que los usuarios emplean para describir cada aspecto en los comentarios, ya que estas palabras suelen dar información valiosa sobre la percepción de los clientes. A partir de esto, crearemos 4 nuevas columnas en nuestro DataFrame:

* adj_adv_COMIDA
* adj_adv_PRECIO
* adj_adv_LOCACIÓN
* adj_adv_SERVICIO

Estas columnas contendrán listas de adjetivos y adverbios que describen cada aspecto, lo que nos permitirá obtener una visión más detallada y cualitativa sobre las opiniones expresadas en los comentarios.

In [6]:
#Copiamos el dataframe que contiene el análisis de sentimiento ya generado
df_adj_adv = df_vader.copy()

In [7]:
# Implementamos el modelo preentrenado 'en_core_web_sm'
nlp1 = spacy.load("en_core_web_sm")

# Creamos la función para extraer los adverbios y adjetivos
def adj_adv(text):
       
        doc = nlp1(text)
        adjectives_adverbs = [token.text for token in doc if token.pos_ in ["ADJ", "ADV"]]

        return adjectives_adverbs




In [8]:
# Creamos una nueva columna de adjetivos y adverbios para cada columna que queremos analizar
df_adj_adv['adj_adv_COMIDA'] = df_adj_adv['COMIDA'].apply(adj_adv)
df_adj_adv['adj_adv_PRECIO'] = df_adj_adv['PRECIO'].apply(adj_adv)
df_adj_adv['adj_adv_LOCACIÓN'] = df_adj_adv['LOCACIÓN'].apply(adj_adv)
df_adj_adv['adj_adv_SERVICIO'] = df_adj_adv['SERVICIO'].apply(adj_adv)

In [9]:
df_adj_adv

,name_x,nombre_estado,city,name_y,rating,text,COMIDA,LOCACIÓN,PRECIO,SERVICIO,COMIDA_SENTIMENT,LOCACIÓN_SENTIMENT,PRECIO_SENTIMENT,SERVICIO_SENTIMENT,adj_adv_COMIDA,adj_adv_PRECIO,adj_adv_LOCACIÓN,adj_adv_SERVICIO
0,Pacific Grove,Massachusetts,Wilmington,Lynn,4,"Very, very good meals. 13 in our party and all...",,"Very, very good meals.",There was 1 employee making sushi and a fair a...,"Very, very good meals.",0.0000,0.5379,0.3182,0.5379,[],"[fair, there]","[Very, very, good]","[Very, very, good]"
1,Confucio Express Brickell,Florida,Miami,Jose David Roa Licenciado en Musica,5,The best Chinese food of Miami I strongly reco...,The best Chinese food of Miami I strongly reco...,,,,0.7964,0.0000,0.0000,0.0000,"[best, Chinese, strongly]",[],[],[]
2,Chinese Chef,Virginia,Henrico,Uma Vadlamani,5,My favorite Chinese food place.,My favorite Chinese food place.,,,,0.4588,0.0000,0.0000,0.0000,"[favorite, Chinese]",[],[],[]
3,Hardee,New York,Brooklyn,wes wright,5,Not your average Chinese spot. Fast service also,Not your average Chinese spot.,,,Fast service also,0.0000,0.0000,0.0000,0.0000,"[average, Chinese]",[],[],"[Fast, also]"
4,Szechuan Delight,New York,Brooklyn,Ralph Parrilla,5,Excellent service very good food could do with...,Excellent service very good food could do with...,,,Excellent service very good food could do with...,0.9509,0.0000,0.0000,0.9509,"[Excellent, very, good, more, excellent]",[],[],"[Excellent, very, good, more, excellent]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,YummyYummy,South Carolina,Great Falls,Scot Simons,4,Food was good and customer service was great!,Food was good and customer service was great!,,,Food was good and customer service was great!,0.8070,0.0000,0.0000,0.8070,"[good, great]",[],[],"[good, great]"
7996,China Kitchen,Delaware,Dover,James Cox,4,"Food was fresh and tasty, I'd say it's above a...","Food was fresh and tasty, I'd say it's above a...",,,,0.6239,0.0000,0.0000,0.0000,"[fresh, tasty, as, far]",[],[],[]
7997,Great Wall Restaurant,South Carolina,Georgetown,Rauf Bolden,5,Excellent chinese cuisine with friendly staff.,,Excellent chinese cuisine with friendly staff.,,,0.0000,0.7845,0.0000,0.0000,[],[],"[Excellent, chinese, friendly]",[]
7998,El Palacio Buffet,Florida,Orlando,Regina A,3,Although diverse Hispanic and Oriental cuisine...,Although diverse Hispanic and Oriental cuisine...,Although diverse Hispanic and Oriental cuisine...,,"Very busy, kind staff and a nice variety of op...",0.3612,0.3612,0.0000,0.7564,"[diverse, Hispanic, Oriental, other]",[],"[diverse, Hispanic, Oriental, other]","[Very, busy, kind, nice]"


### FUTUROS USOS DE NUESTRO MODELO ENTRENADO

**Análisis de Sentimiento Geolocalizado**

* Nuestro modelo ofrece un análisis profundo sobre la percepción de los clientes en restaurantes chinos, segmentado por zona geográfica seleccionada. Esta funcionalidad permite una comprensión detallada de la satisfacción del cliente en cada área, identificando patrones locales clave que ayudan a mejorar la experiencia del cliente y guiar decisiones estratégicas como la expansión, ajustes de precios y optimización de servicios.

**Identificación de Sentimientos Positivos y Negativos**

* La solución clasifica automáticamente las reseñas en positivas o negativas, proporcionando claridad sobre las fortalezas y áreas de mejora percibidas por los clientes. Esto permite a los restaurantes ajustar sus operaciones proactivamente para abordar los puntos débiles y resaltar sus aspectos más valorados, mejorando así la experiencia general del cliente.

**Análisis Detallado y Optimización Basada en Datos**

* El modelo también extrae adjetivos y adverbios clave utilizados por los clientes para describir sus experiencias, ofreciendo un análisis detallado de las emociones expresadas. Combinado con la capacidad de segmentación geográfica, esto facilita la personalización de estrategias de marketing y operaciones. La visualización clara y en tiempo real de estos datos permite una toma de decisiones rápida, basada en hechos y ajustada a las necesidades locales, minimizando el riesgo de actuar sobre suposiciones.